https://huggingface.co/docs/transformers/tasks/sequence_classification

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers datasets torch

Loading the model:

In [23]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline
)
from datasets import load_dataset, load_metric
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
model_name = "roberta-base-openai-detector"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading the dataset:

In [6]:
ott_dataset = load_dataset(
    "csv",
    data_files= "/content/drive/MyDrive/data/ott_et_al_processed.csv",
    delimiter=";"
)

In [7]:
ott_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 1600
})

In [8]:
classifier = pipeline('text-classification', model=model_name)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
predictions = classifier(ott_dataset['train']['text'], max_length=512, truncation=True)

In [11]:
for prediction in predictions:
    print(f"Label: {prediction['label']}, Score: {prediction['score']}")

Label: Real, Score: 0.5432382822036743
Label: Real, Score: 0.9998026490211487
Label: Real, Score: 0.9998146891593933
Label: Real, Score: 0.9998138546943665
Label: Real, Score: 0.9997902512550354
Label: Real, Score: 0.999725878238678
Label: Real, Score: 0.9998247027397156
Label: Real, Score: 0.999828577041626
Label: Real, Score: 0.9998239874839783
Label: Real, Score: 0.9998260140419006
Label: Real, Score: 0.9997791647911072
Label: Real, Score: 0.9997034668922424
Label: Real, Score: 0.9994602799415588
Label: Real, Score: 0.9968396425247192
Label: Real, Score: 0.9997733235359192
Label: Real, Score: 0.9998160004615784
Label: Real, Score: 0.9997946619987488
Label: Fake, Score: 0.992673933506012
Label: Real, Score: 0.999629020690918
Label: Real, Score: 0.9998125433921814
Label: Real, Score: 0.9993017911911011
Label: Real, Score: 0.5626975893974304
Label: Real, Score: 0.9997345805168152
Label: Real, Score: 0.9998076558113098
Label: Real, Score: 0.9998247027397156
Label: Real, Score: 0.9984387

In [13]:
predicted_labels = [prediction['label'] for prediction in predictions]
scores = [prediction['score'] for prediction in predictions]

In [21]:
label_dict = {'Fake': 0, 'Real': 1}  # Replace with your actual label mapping
predicted_labels_numeric = [label_dict[label] for label in predicted_labels]
true_labels = ott_dataset['train']['label']

In [24]:
accuracy = accuracy_score(true_labels, predicted_labels_numeric)
precision = precision_score(true_labels, predicted_labels_numeric, average='binary')  # For binary classification
recall = recall_score(true_labels, predicted_labels_numeric, average='binary')  # For binary classification
f1 = f1_score(true_labels, predicted_labels_numeric, average='binary')  # For binary classification

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.5525
Precision: 0.5295358649789029
Recall: 0.94125
F1 Score: 0.6777677767776777
